In [1]:
import sys
sys.path.append('../')
from utils import BrainGraphDataset, project_root
import torch
import torch.nn as nn
import torch.optim as optim
import os
from torch.utils.data import ConcatDataset
from models import VAE, SameTPred
import copy
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lr = 0.001
batch_size = 32
input_dim = 4950

# define the optimizer and the loss function

criterion = nn.L1Loss(reduction='sum')

root = project_root()

dataroot = 'fc_matrices/psilo_schaefer_before'
annotations = 'annotations-before.csv'
before_dataset = BrainGraphDataset(img_dir=os.path.join(root, dataroot),
                            annotations_file=os.path.join(root, annotations),
                            transform=None, extra_data=None, setting='upper_triangular')

dataroot = 'fc_matrices/psilo_schaefer_after'
annotations = 'annotations-after.csv'
after_dataset = BrainGraphDataset(img_dir=os.path.join(root, dataroot),
                            annotations_file=os.path.join(root, annotations),
                            transform=None, extra_data=None, setting='upper_triangular')

dataset = ConcatDataset([before_dataset, after_dataset])

# Define the train, validation, and test ratios
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

# Get the number of samples in the dataset
num_samples = len(dataset)

# Calculate the number of samples for each set
train_size = int(train_ratio * num_samples)
val_size = int(val_ratio * num_samples)
test_size = num_samples - train_size - val_size

torch.manual_seed(0)
# Split the dataset into train, validation, and test sets
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

test_loader = torch.utils.data.DataLoader(val_set, batch_size=test_size, shuffle=False)

results = []
values = {}
dropout_list = [0, 0.05 ,0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for dropout in dropout_list:

    vae = VAE(4950, [128] * 2, 64)
    vae.load_state_dict(torch.load(os.path.join(root, 'same_t_pred', f'vae_unfrozen_dropout_{dropout}.pt'), 
                                   map_location=device))
    vae = vae.to(device)

    model = SameTPred(64, 256, dropout=dropout)
    model.load_state_dict(torch.load(os.path.join(root, 'same_t_pred', f'same_t_weight_dropout_{dropout}.pt'), 
                          map_location=device))

    # Convert the MLP to the device
    model.to(device)

    # Validation check
    val_loss = 0.0
    model.eval()
    vae.eval()
    with torch.no_grad():
        for data in test_loader:
            graphs, labels = data

            graphs = graphs.to(device)
            labels = labels.to(device).float()

            _, _, _, z = vae(graphs.view(-1, input_dim))

            preds = model(z)

            labels = labels.view(-1).cpu().numpy()
            preds = preds.view(-1).cpu().numpy()
            
            mae = mean_absolute_error(labels, preds)
            corr_coeffs = pearsonr(labels, preds)
            r2 = r2_score(labels, preds)

            results.append((dropout, mae, corr_coeffs[0], corr_coeffs[1], r2))
            values[f'{dropout}'] = {'ground': labels.tolist(), 'predicted': preds.tolist()}
            
import csv
import json

with open(os.path.join(root, 'same_t_pred', 'test_results.json'), 'w') as f:
    json.dump(values, f)

csv_filename = os.path.join(root, 'same_t_pred', 'same_t_dropout_results.csv')
# Write the results to the CSV file
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(["Dropout", "MAE", "Pearson stat", "Pearson p-value", "R2 Score"])
    
    # Write the data rows
    for result in results:
        writer.writerow(result)